In [66]:
#以下、ルールベース用　annotated_playdata.csvの前処理

#縦位置、横位置、方向8、タッチ種別をそれぞれワンホットベクトルに変換
#あつかうコラムを以下のもののみとする
# ['試合ID', '攻撃履歴No', 'チームID', '選手ID', '選手名', 'チーム名', '縦位置_1', '縦位置_2',
#        '縦位置_3', '縦位置_4', '縦位置_5', '縦位置_6', '横位置_1', '横位置_2', '横位置_3', '横位置_4',
#        '方向8_前', '方向8_右', '方向8_右前', '方向8_右後', '方向8_左', '方向8_左前', '方向8_左後',
#        '方向8_後', 'タッチ種別_0', 'タッチ種別_1', 'タッチ種別_2', 'タッチ種別_3', 'F_成功', 'F_セットプレー',
#        'F_ゴール', 'F_シュート', 'F_ミスヒット', 'F_アシスト', 'F_シュートアシスト', 'F_パス', 'F_クロス',
#        'F_スルーパス', 'F_フリックオン', 'F_フィード', 'F_パス受け', 'F_トラップ', 'F_ドリブル',
#        'F_ペナルティエリア進入', 'F_ペナルティ脇進入', 'F_30mライン進入', 'F_こぼれ球奪取', 'F_クリア',
#        'F_インターセプト', 'F_ブロック', 'F_タックル', '空中戦F', 'F_キャッチ', 'F_ファンブル',
#        'F_ハンドクリア', 'F_ゴールキック', 'F_コーナーキック', 'F_直接フリーキック', 'F_間接フリーキック',
#        'F_ペナルティキック', 'F_スローイン', 'F_ファウル', 'F_PK与えた', 'F_被ファウル', 'F_オフサイド',
#        'F_イエローカード', 'F_レッドカード', 'F_ポスト・バー', 'パスToWho', '部位ID', 'ポジションID',
#        '相手選手ID', 'アクション名', 'comment', 'comment_golden']

#comment=Placeholderあり、comment_golden=Placeholderなし


In [74]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import MeCab
import random
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset
import warnings
import math

warnings.filterwarnings("ignore", category=FutureWarning)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
df = pd.read_csv('/path/to/annotated_playdata.csv')

for column in ['縦位置','横位置','方向8','タッチ種別']:
    df_dummy = pd.get_dummies(df[column])
    df_dummy.columns = [f"{column}_{col}" for col in df_dummy.columns]
    df = pd.concat([df.drop([column],axis=1),df_dummy], axis=1)

df = df[['試合ID', '攻撃履歴No', 'チームID', '選手ID', '選手名', 'チーム名',  '縦位置_1', '縦位置_2',
       '縦位置_3', '縦位置_4', '縦位置_5', '縦位置_6', '横位置_1', '横位置_2', '横位置_3', '横位置_4',
       '方向8_前', '方向8_右', '方向8_右前', '方向8_右後', '方向8_左', '方向8_左前', '方向8_左後',
       '方向8_後', 'タッチ種別_0', 'タッチ種別_1', 'タッチ種別_2', 'タッチ種別_3', 'F_成功', 'F_セットプレー',
       'F_ゴール', 'F_シュート', 'F_ミスヒット', 'F_アシスト', 'F_シュートアシスト', 'F_パス', 'F_クロス',
       'F_スルーパス', 'F_フリックオン', 'F_フィード', 'F_パス受け', 'F_トラップ', 'F_ドリブル',
       'F_ペナルティエリア進入', 'F_ペナルティ脇進入', 'F_30mライン進入', 'F_こぼれ球奪取', 'F_クリア',
       'F_インターセプト', 'F_ブロック', 'F_タックル', '空中戦F', 'F_キャッチ', 'F_ファンブル',
       'F_ハンドクリア', 'F_ゴールキック', 'F_コーナーキック', 'F_直接フリーキック', 'F_間接フリーキック',
       'F_ペナルティキック', 'F_スローイン', 'F_ファウル', 'F_PK与えた', 'F_被ファウル', 'F_オフサイド',
       'F_イエローカード', 'F_レッドカード', 'F_ポスト・バー', 'パスToWho', '部位ID', 'ポジションID',
       '相手選手ID', 'アクション名', 'comment', 'comment_golden']]
df.columns

Index(['試合ID', '攻撃履歴No', 'チームID', '選手ID', '選手名', 'チーム名', '縦位置_1', '縦位置_2',
       '縦位置_3', '縦位置_4', '縦位置_5', '縦位置_6', '横位置_1', '横位置_2', '横位置_3', '横位置_4',
       '方向8_前', '方向8_右', '方向8_右前', '方向8_右後', '方向8_左', '方向8_左前', '方向8_左後',
       '方向8_後', 'タッチ種別_0', 'タッチ種別_1', 'タッチ種別_2', 'タッチ種別_3', 'F_成功', 'F_セットプレー',
       'F_ゴール', 'F_シュート', 'F_ミスヒット', 'F_アシスト', 'F_シュートアシスト', 'F_パス', 'F_クロス',
       'F_スルーパス', 'F_フリックオン', 'F_フィード', 'F_パス受け', 'F_トラップ', 'F_ドリブル',
       'F_ペナルティエリア進入', 'F_ペナルティ脇進入', 'F_30mライン進入', 'F_こぼれ球奪取', 'F_クリア',
       'F_インターセプト', 'F_ブロック', 'F_タックル', '空中戦F', 'F_キャッチ', 'F_ファンブル',
       'F_ハンドクリア', 'F_ゴールキック', 'F_コーナーキック', 'F_直接フリーキック', 'F_間接フリーキック',
       'F_ペナルティキック', 'F_スローイン', 'F_ファウル', 'F_PK与えた', 'F_被ファウル', 'F_オフサイド',
       'F_イエローカード', 'F_レッドカード', 'F_ポスト・バー', 'パスToWho', '部位ID', 'ポジションID',
       '相手選手ID', 'アクション名', 'comment', 'comment_golden'],
      dtype='object')

In [75]:
# 各行の中で値が1である列名を取得する関数
def get_column_names_with_value_1(row):
    return [str(column.replace("F_", "")) for column, value in row.items() if value == 1]

def get_location(event):
    if (event['縦位置_1'] == 1) and (event['横位置_1'] == 1) :
        return '左サイド敵陣深く'
    if (event['縦位置_1'] == 1) and ((event['横位置_2'] == 1) or (event['横位置_3'] == 1)):
        return 'ペナルティエリア内'
    if (event['縦位置_1'] == 1) and (event['横位置_4'] == 1) :
        return '右サイド敵陣深く'
    if ((event['縦位置_2'] == 1)or(event['縦位置_3'] == 1)) and (event['横位置_1'] == 1) :
        return '左サイド敵陣'
    if ((event['縦位置_2'] == 1)or(event['縦位置_3'] == 1)) and ((event['横位置_2'] == 1) or (event['横位置_3'] == 1)) :
        return '敵陣中央'
    if ((event['縦位置_2'] == 1)or(event['縦位置_3'] == 1)) and (event['横位置_4'] == 1) :
        return '右サイド敵陣'
    if ((event['縦位置_4'] == 1)or(event['縦位置_5'] == 1) or (event['縦位置_6'] == 1)) and (event['横位置_1'] == 1) :
        return '左サイド味方陣内'
    if ((event['縦位置_4'] == 1)or(event['縦位置_5'] == 1)) and ((event['横位置_2'] == 1) or (event['横位置_3'] == 1)) :
        return '味方陣内中央'
    if ((event['縦位置_4'] == 1)or(event['縦位置_5'] == 1)or (event['縦位置_6'] == 1)) and (event['横位置_4'] == 1) :
        return '右サイド味方陣内'
    if (event['縦位置_6'] == 1) and ((event['横位置_2'] == 1) or (event['横位置_3'] == 1)):
        return 'ペナルティエリア内'

In [69]:
#各プレーの選抜コラムたち
def setplay_action(actions):
    if 'ゴールキック' in actions :
        return 'ゴールキック'
    
    elif 'コーナーキック' in actions :
        return 'コーナーキック'
    
    elif '直接フリーキック' in actions :
        return '直接フリーキック'
    
    elif '間接フリーキック' in actions :
        return '間接フリーキック'
    
    elif 'ペナルティキック' in actions :
        return 'ペナルティキック'
    
    elif 'スローイン' in actions :
        return 'スローイン'
    
    else:
        return None
    
def pass_action(actions):  
    if 'クロス' in actions :
        return 'クロス'
    
    elif 'スルーパス' in actions :
        return 'スルーパス'
    
    elif 'フリックオン' in actions :
        return 'フリックパス'
    
    elif 'フィード' in actions :
        return 'フィード'
    
    else:
        return 'パス'
    

    

In [76]:
player_dict = df.set_index('選手ID')['選手名'].to_dict()
player_dict[1607284] = '松本'

position_dict = {-1:'選手詳細',1:'ＧＫ',2:'ＤＦ',3:'ＭＦ',4:'ＦＷ'}
point_dict = {1:'左足',2:'右足',3:'頭'}

In [77]:
# 列名の頭文字が大文字 "F" で始まる列を抽出
F_columns = [column for column in df.columns if column[0] == 'F']
F_columns = [item for item in F_columns if item != 'F_成功']
df_F = df[F_columns]
df['actions'] = df_F.apply(get_column_names_with_value_1, axis=1)
df.head()

,試合ID,攻撃履歴No,チームID,選手ID,選手名,チーム名,縦位置_1,縦位置_2,縦位置_3,縦位置_4,...,F_レッドカード,F_ポスト・バー,パスToWho,部位ID,ポジションID,相手選手ID,アクション名,comment,comment_golden,actions
0,2021040201,6,269,1400253,朴,鳥栖,False,False,False,False,...,0,0,0,0,1,0,トラップ,team 低い位置からパスをつなぎ、徐々に前線へボールを運ぶが、player1のボールコント...,鳥栖 低い位置からパスをつなぎ、徐々に前線へボールを運ぶが、中野伸のボールコントロールがずれ...,[トラップ]
1,2021040201,6,269,1400253,朴,鳥栖,False,False,False,False,...,0,0,1200083,0,1,0,アウェイパス,team 低い位置からパスをつなぎ、徐々に前線へボールを運ぶが、player1のボールコント...,鳥栖 低い位置からパスをつなぎ、徐々に前線へボールを運ぶが、中野伸のボールコントロールがずれ...,[パス]
2,2021040201,6,269,1200083,ファンソッコ,鳥栖,False,False,False,False,...,0,0,0,0,2,0,トラップ,team 低い位置からパスをつなぎ、徐々に前線へボールを運ぶが、player1のボールコント...,鳥栖 低い位置からパスをつなぎ、徐々に前線へボールを運ぶが、中野伸のボールコントロールがずれ...,"[パス受け, トラップ]"
3,2021040201,6,269,1200083,ファンソッコ,鳥栖,False,False,False,False,...,0,0,1300886,0,2,0,アウェイパス,team 低い位置からパスをつなぎ、徐々に前線へボールを運ぶが、player1のボールコント...,鳥栖 低い位置からパスをつなぎ、徐々に前線へボールを運ぶが、中野伸のボールコントロールがずれ...,[パス]
4,2021040201,6,269,1300886,エドゥアルド,鳥栖,False,False,False,False,...,0,0,0,0,2,0,トラップ,team 低い位置からパスをつなぎ、徐々に前線へボールを運ぶが、player1のボールコント...,鳥栖 低い位置からパスをつなぎ、徐々に前線へボールを運ぶが、中野伸のボールコントロールがずれ...,"[パス受け, トラップ]"


In [78]:
def make_rulebase_comment(df):
    comments = []
    faul_player = []
    action_list = []
    goal = 0
    #print(df.iloc[0,:]['試合ID'], df.iloc[0,:]['攻撃履歴No'])
    for i in range(len(df)):
        match_id1 = df.iloc[i,:]['試合ID']
        attack_history1 = df.iloc[i,:]['攻撃履歴No']
        print(f'試合ID： {match_id1}, 攻撃履歴No：{attack_history1}')
        player = df.iloc[i,:]['選手名']
        location = get_location(df.iloc[i,:])
        position_key = df.iloc[i,:]['ポジションID']
        team = df.iloc[i,:]['チーム名']
        actions = df.iloc[i,:]['actions']
        pass_to_who = df.iloc[i,:]['パスToWho']
        success = '成功' if df.iloc[i,:]['F_成功'] == 1 else '失敗'
        
    
        #(シュートの場合もあるがセットプレーを優先すべき)セットプレーのルールベース文、発動条件：F_セットプレーが1
        #入れたい情報：誰がどこから、セットプレーをして、
        #パスの場合、誰に出して、それが成功したか
        #シュートの場合、シュートをしてそれがミスヒットしたか、ゴールしたか
        
        if df.iloc[i,:]['F_セットプレー'] == 1:
            position = position_dict[position_key]
            point_id = df.iloc[i,:]['部位ID']
            action = setplay_action(actions)
            action_list.append(action)
            if (point_id != -1) and (point_id != 0)  and (point_id != 4):
                point = point_dict[point_id]
                if df.iloc[i,:]['F_パス'] == 1:
                    
                    pass_to_who = player_dict[pass_to_who]
                    if pass_to_who == 'missing':
                        if success == '成功':
                            if action != 'スローイン':
                                role_base_text = f'{team}の{position}{player}が{location}から{point}で{action}をした。'
                            else:
                                role_base_text = f'{team}の{player}が{location}からスローインを投げた。'
                        else:
                            if action != 'スローイン':
                                role_base_text = f'{team}の{position}{player}が{location}から{point}で{action}をするもボールは通らない。'
                            else:
                                role_base_text = f'{team}の{player}が{location}からスローインをするもボールは通らない。'
                    else:
                        
                        if success == '成功':
                            if action != 'スローイン':
                                role_base_text = f'{team}の{position}{player}が{location}から{pass_to_who}へ{point}で{action}をした。'
                            else:
                                role_base_text = f'{team}の{player}が{location}から{pass_to_who}へスローインを投げた。'
                                
                        else:
                            if action != 'スローイン':
                                role_base_text = f'{team}の{position}{player}が{location}から{pass_to_who}へ{point}で{action}をするもパスは通らない。'
                            else:
                                role_base_text = f'{team}の{player}が{location}から{pass_to_who}へスローインをするもパスは通らない。'
                    
                elif df.iloc[i,:]['F_シュート'] == 1:
                    if df.iloc[i,:]['F_ゴール'] == 1:
                        role_base_text = f'{team}の{position}{player}が{location}から{point}の{action}で直接ゴールを狙い、見事にゴールを決めた。' 
                    else:
                        role_base_text = f'{team}の{position}{player}が{location}から{point}の{action}で直接ゴールを狙ったがゴールを決めることはできない。' 
            else:
                if df.iloc[i,:]['F_パス'] == 1:
                    pass_to_who = player_dict[pass_to_who]
                    if pass_to_who == 'missing':
                        if success == '成功':
                            role_base_text = f'{team}の{position}{player}が{location}から{action}をした。'
                        else:
                            role_base_text = f'{team}の{position}{player}が{location}から{action}をするもパスは通らない。'
                    else:
                        if success == '成功':
                            role_base_text = f'{team}の{position}{player}が{location}から{pass_to_who}へ{action}をした。'
                        else:
                            role_base_text = f'{team}の{position}{player}が{location}から{pass_to_who}へ{action}をするもパスは通らない。'
                    
                elif df.iloc[i,:]['F_シュート'] == 1:
                    if df.iloc[i,:]['F_ゴール'] == 1:
                        role_base_text = f'{team}の{position}{player}が{location}から{action}で直接ゴールを狙い、見事にゴールを決めた。' 
                    else:
                        role_base_text = f'{team}の{position}{player}が{location}から{action}で直接ゴールを狙ったがゴールを決めることはできない。' 
                
        
        #ポジションIDから「どのポジションの」の情報も全てに入れたほうがいい
        #パスのルールベース文、発動条件：F_パスが1        
        #入れたい情報：誰がどこから、どの方向へ、誰にどんなパスをして、それが成功したか
        #どんなパス：F_クロス、F_スルーパスなど特殊パスが来たらそれを優先、それらがすべて0ならアクションには’パス’が入る
        
        elif df.iloc[i,:]['F_パス'] == 1:
            point_id = df.iloc[i,:]['部位ID']
            pass_to_who = player_dict[pass_to_who]
            if (point_id != -1) and (point_id != 0)  and (point_id != 4):
                point = point_dict[point_id]
                if pass_to_who == 'missing':
                    position = position_dict[position_key]
                    action = pass_action(actions)
                    action_list.append(action)
                    if success == '成功':
                        role_base_text = f'{team}の{position}{player}が{location}から{point}で{action}をした。'
                    else:
                        role_base_text = f'{team}の{position}{player}が{location}から{point}で{action}をするもボールは通らない。'
                else:
                    position = position_dict[position_key]
                    action = pass_action(actions)
                    action_list.append(action)
                    if success == '成功':
                        role_base_text = f'{team}の{position}{player}が{location}から{pass_to_who}へ{point}で{action}をした。'
                    else:
                        role_base_text = f'{team}の{position}{player}が{location}から{pass_to_who}へ{point}で{action}をするもボールは通らない。'
            else:
                if pass_to_who == 'missing':
                    position = position_dict[position_key]
                    action = pass_action(actions)
                    action_list.append(action)
                    if success == '成功':
                        role_base_text = f'{team}の{position}{player}が{location}から{action}をした。'
                    else:
                        role_base_text = f'{team}の{position}{player}が{location}から{action}をするもボールは通らない。'
                else:
                    position = position_dict[position_key]
                    action = pass_action(actions)
                    action_list.append(action)
                    if success == '成功':
                        role_base_text = f'{team}の{position}{player}が{location}から{pass_to_who}へ{action}をした。'
                    else:
                        role_base_text = f'{team}の{position}{player}が{location}から{pass_to_who}へ{action}をするもボールは通らない。'
        
            
        #シュートのルールベース文、発動条件：F_シュートが1
        #入れたい情報：誰がどこから、シュートをして、それが成功したか
        #シュートをしてそれがミスヒットしたか、ゴールしたかポストバーか
        elif df.iloc[i,:]['F_シュート'] == 1:
            position = position_dict[position_key]
            point = point_dict[df.iloc[i,:]['部位ID']]
            action = 'シュート'
            action_list.append(action)
            
            if success == '成功':
                role_base_text = f'{team}の{position}{player}が{location}から{point}でシュートをして、ゴールを決めた。'
                goal += 1
            else:
                role_base_text = f'{team}の{position}{player}が{location}から{point}でシュートをするもゴールには至らず。'
            
        #ポスト・バーのルールベース文、発動条件：F_ポストバーが1
        #入れたい情報：誰の（一つ前の人を参照？）シュートがポストにあたった   
        elif df.iloc[i,:]['F_ポスト・バー'] == 1:
            role_base_text = f'ボールがゴールポストに直撃する。'
        
        #ファウルのルールベース文、発動条件：F_ファウルが1
        #入れたい情報：誰がどこで、ファウルしたか（されたか）
        #された側、した側、それぞれの文を生成
        # elif df.iloc[i,:]['F_PK与えた'] == 1:
        #     faul_player.append(player)
        #     position = position_dict[position_key]
            
        #     role_base_text = f'{team}の{position}{player}が{location}でファウルしPKを与えた。'
            
        # elif df.iloc[i,:]['F_被ファウル'] == 1:
        #     position = position_dict[position_key]
        #     role_base_text = f'{team}の{position}{player}が{location}で{faul_player[0]}にファウルされた。'
        
        # elif df.iloc[i,:]['F_ファウル'] == 1:
        #     faul_player.append(player)
        #     position = position_dict[position_key]
        #     if df.iloc[i,:]['F_イエローカード'] == 1:
        
        #         if df.iloc[i,:]['F_PK与えた'] == 1:
        #             role_base_text = f'{team}の{position}{player}が{location}でファウルしイエローカードをもらい、PKを与えた。'
        #         else:
        #             role_base_text = f'{team}の{position}{player}が{location}でファウルしてイエローカードをもらった。'
                    
        #     elif df.iloc[i,:]['F_レッドカード'] == 1:
            
        #         if df.iloc[i,:]['F_PK与えた'] == 1:
        #             role_base_text = f'{team}の{position}{player}が{location}でファウルしレッドカードをもらい、PKを与えた。'
        #         else:
        #             role_base_text = f'{team}の{position}{player}が{location}でファウルしてレッドカードをもらった。'
        #     else:
        #         role_base_text = f'{team}の{position}{player}が{location}でファウルした。'
        elif df.iloc[i,:]['F_ファウル'] == 1:
            faul_player.append(player)
            yellow = 0
            red = 0
            pk = 0
            if (i + 1) == len(df):
                if df.iloc[i,:]['F_イエローカード'] == 1:
                    if df.iloc[i,:]['F_PK与えた'] == 1:
                        role_base_text = f'{team}の{position}{player}が{location}でファウルしPKを与える。{player}はイエローカードの判定。'
                    else:
                        role_base_text = f'{team}の{position}{player}が{location}でファウルしイエローカードをもらう。'
                        
                elif df.iloc[i,:]['F_レッドカード'] == 1:
                    if df.iloc[i,:]['F_PK与えた'] == 1:
                        role_base_text = f'{team}の{position}{player}が{location}でファウルしPKを与える。{player}はレッドカードの判定。'
                    else:
                        role_base_text = f'{team}の{position}{player}が{location}でファウルしレッドカードをもらう。'
                
                else:
                    role_base_text = f'{team}の{position}{player}が{location}でファウルしてしまう。'  
                        
            else:
                if df.iloc[i,:]['F_イエローカード'] == 1:
                    yellow = 1
                if df.iloc[i,:]['F_レッドカード'] == 1:
                    red = 1
                if df.iloc[i,:]['F_PK与えた'] == 1:
                    pk = 1
                role_base_text = ''
        
        elif df.iloc[i,:]['F_被ファウル'] == 1:
            position = position_dict[position_key]
            if yellow == 1:
                if pk == 1:
                    role_base_text = f'{team}の{position}{player}が{location}で{faul_player[0]}に倒されPKを得る。{faul_player[0]}はイエローカードの判定。'
                else:
                    role_base_text = f'{team}の{position}{player}が{location}で{faul_player[0]}に倒された。{faul_player[0]}はイエローカードの判定。'
                    
            elif red == 1:
                if pk == 1:
                    role_base_text = f'{team}の{position}{player}が{location}で{faul_player[0]}に倒されPKを得る。{faul_player[0]}はレッドカードの判定。'
                else:
                    role_base_text = f'{team}の{position}{player}が{location}で{faul_player[0]}に倒された。{faul_player[0]}はレッドカードの判定。' 
            else:
                if pk == 1:
                    role_base_text = f'{team}の{position}{player}が{location}で{faul_player[0]}に倒されPKを得る。'
                else:
                    role_base_text = f'{team}の{position}{player}が{location}で{faul_player[0]}に倒された。'
            
                
        
                
            
        elif df.iloc[i,:]['F_オフサイド'] == 1:
            position = position_dict[position_key]
            
            role_base_text = f'{team}の{position}{player}がオフサイドをした。'
            
        
        #ドリブルのルールベース文、発動条件：F_ドリブルが1
        #入れたい情報：誰がどこで、ドリブルし、（もししていたらペナルティエリアに侵入したか）成功したか
        elif df.iloc[i,:]['F_ドリブル'] == 1:
            position = position_dict[position_key]
            if df.iloc[i,:]['F_ペナルティ脇進入'] == 1:
                role_base_text = f'{team}の{position}{player}が{location}でドリブルしペナルティエリア脇に侵入した。'
            
            elif df.iloc[i,:]['F_ペナルティエリア進入'] == 1:
                role_base_text = f'{team}の{position}{player}が{location}でドリブルしペナルティエリアに侵入した。'
                
            else:
                if success == '成功':
                    role_base_text = f'{team}の{position}{player}が{location}でドリブルをし相手をかわした。'
                else:
                    role_base_text = f'{team}の{position}{player}が{location}でドリブルをするも相手にボールを奪われた。'
            
        #インターセプトのルールベース文、発動条件：F_インターセプトが1
        #入れたい情報：誰がどこで、インターセプトしたか
        
        elif df.iloc[i,:]['F_インターセプト'] == 1:
            position = position_dict[position_key]
            role_base_text = f'{team}の{position}{player}が{location}で相手のパスをインターセプトした。'
        
        #クリアのルールベース文、発動条件：F_クリアが1
        #入れたい情報：誰がどこで、どこにクリアしたか
        elif df.iloc[i,:]['F_クリア'] == 1:
            position = position_dict[position_key]
            role_base_text = f'{team}の{position}{player}が{location}からボールをクリアした。'
        
        #ブロックのルールベース文、発動条件：F_ブロックが1
        #入れたい情報：誰がどこで、ブロックしたか
        elif df.iloc[i,:]['F_ブロック'] == 1:
            position = position_dict[position_key]
            if i != 0:
                role_base_text = f'{team}の{position}{player}が{location}で相手の{action_list[-1]}をブロックした。'
            else:
                role_base_text = f'{team}の{position}{player}が{location}で相手のボールをブロックした。'
                
        #タックルのルールベース文、発動条件：F_タックルが1
        #入れたい情報：誰がどこで、誰に？タックルして成功したか
        elif df.iloc[i,:]['F_タックル'] == 1:
            position = position_dict[position_key]
            opponent = df.iloc[i,:]['相手選手ID']
            opponent = player_dict[opponent]
            if success == '成功':
                role_base_text = f'{team}の{position}{player}が{location}で相手チームの{opponent}にタックルをしてボールを奪った。'
            else:
                role_base_text = f'{team}の{position}{player}が{location}で相手チームの{opponent}にタックルをするもボールは奪えない。'
        #空中線のルールベース文、発動条件：空中線Fが1
        #入れたい情報：誰がどこで、誰と空中戦をして成功したか
        elif df.iloc[i,:]['空中戦F'] == 1:
            position = position_dict[position_key]
            opponent = df.iloc[i,:]['相手選手ID']
            if success == '成功':
                role_base_text = f'{team}の{position}{player}が{location}で{opponent}と空中戦を行い競り勝った。'
            else:
                role_base_text = f'{team}の{position}{player}が{location}で{opponent}と空中戦を行い競り負けた。'   
        #キーパーのルールベース文、発動条件：ポジションIDが1
        #入れたい情報：GKの誰が何をしたか


        elif df.iloc[i,:]['F_キャッチ'] == 1:
            position = position_dict[position_key]
            if df.iloc[i,:]['F_ファンブル'] == 1:
                if i != 0:
                    role_base_text = f'{team}の{position}{player}が相手の{action_list[-1]}をキャッチしようとするも、ファンブルしてしまう。'
                else:
                    role_base_text = f'{team}の{position}{player}が相手のボールをキャッチしようとするも、ファンブルしてしまう。'
            else:
                if i != 0:
                    role_base_text = f'{team}の{position}{player}が相手の{action_list[-1]}をキャッチした。'
                else:
                    role_base_text = f'{team}の{position}{player}が相手のボールをキャッチした。'
            
        elif df.iloc[i,:]['F_ハンドクリア'] == 1:
            position = position_dict[position_key]
            if success == '成功':
                role_base_text = f'{team}の{position}{player}が相手の{action_list[-1]}をハンドクリアした。'  
            else:
                role_base_text = f'{team}の{position}{player}が相手の{action_list[-1]}をハンドクリアしようとしたが手を弾きボールはゴールの中へ。'  
        
        #（正直これはあまり枠）トラップのルールベース文、発動条件：F_トラップのみが1
        #入れたい情報：誰がどこで、トラップして成功したか
        
        elif df.iloc[i,:]['F_トラップ'] == 1:
            position = position_dict[position_key]
            if success == '成功':
                role_base_text = f'{team}の{position}{player}が{location}でボールをトラップした。'
            else:
                role_base_text = f'{team}の{position}{player}が{location}でボールをトラップしようとしたがうまく収まらない。'
                
        elif df.iloc[i,:]['アクション名'] == 'ボールアウト':
            if goal == 1:
                role_base_text = ''
            else:
                role_base_text = f'ボールアウトしてしまう。'
                
        else:
            role_base_text = f'{team}の{position}{player}がオウンゴールをしてしまう。'
        
        comments.append(role_base_text)
    comments = ''.join(comments)
    return comments

In [79]:
#新しいデータフレームを作成し、そこにinput,outputを攻撃履歴ごとに入れていく。
input = []
output = []
for match in sorted(df['試合ID'].unique()):
    df_match = df[df['試合ID'] == match]
    for attack_history in sorted(df_match['攻撃履歴No'].unique()):
        events = df_match[df_match['攻撃履歴No'] == attack_history]
        input.append(make_rulebase_comment(events))
        output.append(events.iloc[0,:]['comment_golden'])

df_rulebase_input_output = {'input': input, 'output': output}
df_rulebase_input_output = pd.DataFrame(df_rulebase_input_output)
df_rulebase_input_output.head()

試合ID： 2021040201, 攻撃履歴No：6
試合ID： 2021040201, 攻撃履歴No：6
試合ID： 2021040201, 攻撃履歴No：6
試合ID： 2021040201, 攻撃履歴No：6
試合ID： 2021040201, 攻撃履歴No：6
試合ID： 2021040201, 攻撃履歴No：6
試合ID： 2021040201, 攻撃履歴No：6
試合ID： 2021040201, 攻撃履歴No：6
試合ID： 2021040201, 攻撃履歴No：6
試合ID： 2021040201, 攻撃履歴No：6
試合ID： 2021040201, 攻撃履歴No：6
試合ID： 2021040201, 攻撃履歴No：6
試合ID： 2021040201, 攻撃履歴No：6
試合ID： 2021040201, 攻撃履歴No：6
試合ID： 2021040201, 攻撃履歴No：6
試合ID： 2021040201, 攻撃履歴No：6
試合ID： 2021040201, 攻撃履歴No：6
試合ID： 2021040201, 攻撃履歴No：6
試合ID： 2021040201, 攻撃履歴No：6
試合ID： 2021040201, 攻撃履歴No：6
試合ID： 2021040201, 攻撃履歴No：6
試合ID： 2021040201, 攻撃履歴No：10
試合ID： 2021040201, 攻撃履歴No：10
試合ID： 2021040201, 攻撃履歴No：10
試合ID： 2021040201, 攻撃履歴No：10
試合ID： 2021040201, 攻撃履歴No：20
試合ID： 2021040201, 攻撃履歴No：20
試合ID： 2021040201, 攻撃履歴No：20
試合ID： 2021040201, 攻撃履歴No：24
試合ID： 2021040201, 攻撃履歴No：24
試合ID： 2021040201, 攻撃履歴No：24
試合ID： 2021040201, 攻撃履歴No：24
試合ID： 2021040201, 攻撃履歴No：24
試合ID： 2021040201, 攻撃履歴No：24
試合ID： 2021040201, 攻撃履歴No：24
試合ID： 2021040201, 攻撃履歴No：24
試合ID： 2021040

,input,output
0,鳥栖のＧＫ朴がペナルティエリア内でボールをトラップした。鳥栖のＧＫ朴がペナルティエリア内から...,鳥栖 低い位置からパスをつなぎ、徐々に前線へボールを運ぶが、中野伸のボールコントロールがずれ...
1,Ｃ大阪のＭＦ西川が右サイド敵陣から河井へ左足で直接フリーキックをした。Ｃ大阪のＦＷ豊川がペナ...,Ｃ大阪 右サイドの敵陣中央でＦＫを獲得する。キッカーの西川は左足でクロスを上げると、豊川がヘ...
2,鳥栖のＤＦエドゥアルドが味方陣内中央でボールをトラップした。鳥栖のＤＦ中野嘉が敵陣中央から吉...,鳥栖 エドゥアルドが敵陣浅い位置からスルーパスを供給すると、中野嘉が敵陣中央で受ける。しかし...
3,鳥栖のＤＦ仙頭が左サイド味方陣内から小泉へパスをした。鳥栖のＭＦ仙頭が左サイド味方陣内でボー...,鳥栖 飯野が右サイドからドリブルを仕掛けると、右サイドの敵陣深くからクロスを上げる。しかし、...
4,鳥栖のＤＦ松岡が右サイド敵陣深くから岩波へスローインをした。鳥栖のＭＦ松岡が右サイド敵陣でボ...,鳥栖 仙頭がペナルティエリア左からスルーパスを出すと、小屋松がゴールライン際で受ける。しかし...


In [80]:
df_rulebase_input_output.to_csv('rulebase_dataset.csv', index= False, encoding='utf-8')